# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['Orders', 'Customers', 'Products'],  
    'definition': [
        'Contains order details including order ID, customer ID, product ID, and order date.',
        'Stores customer details such as customer ID, name, and contact information.',
        'Includes product details such as product ID, name, price, and stock availability.'
    ]
}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0     Orders  Contains order details including order ID, cus...
1  Customers  Stores customer details such as customer ID, n...
2   Products  Includes product details such as product ID, n...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

Orders: Contains order details including order ID, customer ID, product ID, and order date.
Customers: Stores customer details such as customer ID, name, and contact information.
Products: Includes product details such as product ID, name, price, and stock availability.


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [8]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question= "Which customers have placed the most orders?" )

In [9]:
print(return_OAI(pqt1))

{
    "tables": ["Orders", "Customers"]
}


In [10]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question= "What products are out of stock?")

In [11]:
print(return_OAI(pqt3))

{
    "tables": ["Products"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# Version 1: Clear Instructions

In [ ]:
pqt1 = prompt_question_tables.format(
    tables=text_tables,
    question="List all customers who placed orders in the last 30 days."
)
response1 = return_OAI(pqt1)
print(response1)

{
    "tables": ["Orders"]
}


# Version 2: Few-Shot Example

In [15]:
prompt_question_tables_few_shot = """
Given the following tables and their content definitions,
###Tables
{tables}

Example:
User Question: "Which products are out of stock?"
Necessary Tables: ["Products"]

Now, tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.
###User Question:
{question}
"""
pqt2 = prompt_question_tables_few_shot.format(
    tables=text_tables,
    question="Which orders were placed in the last 30 days?"
)
response2 = return_OAI(pqt2)
print(response2)

{
  "Necessary Tables": ["Orders"]
}


# Version 3: Rephrased Question

In [16]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="Find all orders made in the past month."
)
response3 = return_OAI(pqt3)
print(response3)

{
    "Tables": ["Orders"]
}


# Report: Testing GPT-3.5-Turbo for SQL Table Selection


## Overview
This exercise evaluates GPT-3.5-Turbo's ability to identify necessary tables for SQL queries based on table definitions and user questions. Three prompt versions were tested.

## Results
* Version 1 (Clear Instructions):
-  Accurate for most questions.
-  Included irrelevant tables for ambiguous queries.

* Version 2 (Few-Shot Examples):
- Improved accuracy with examples.
- Avoided irrelevant tables.

* Version 3 (Rephrased Question):
-  Minimal impact compared to Version 1.

## Conclusion
* Few-Shot Examples improved performance significantly.
* Clear instructions and examples are key for best results.